In [2]:
import redis
from time import time
from datetime import date, datetime

REDIS_HOST = "redis-18937.c72.eu-west-1-2.ec2.cloud.redislabs.com"
REDIS_PORT = 18937
REDIS_USER = "default"
REDIS_PASSWORD = "DlfmUPWr2iMKAbzvEiwzLCizwt2yjLkP"

# establish a connection to redis
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USER, password=REDIS_PASSWORD)

# politecnico wifi block redis connections
print('Is connected: ', redis_client.ping())

Is connected:  True


In [11]:
mac_address = "0x38baf82a5cb6"
battery_string = f'{mac_address}:battery'
power_string = f'{mac_address}:power'
# power_plugged_seconds_string = f'{mac_address}:plugged_seconds'
plugged_seconds_string = "0x38baf82a5cb6:plugged_seconds"

In [32]:
# create 2 timestamp to read data from a window
from_datetime = datetime.fromisoformat('2022-10-31 17:00:00')
from_timestamp_in_s = from_datetime.timestamp()
from_timestamp_in_ms = int(from_timestamp_in_s * 1000)
print(from_timestamp_in_ms)

to_datetime = datetime.fromisoformat('2022-11-01 06:00:00')
to_timestamp_in_s = to_datetime.timestamp()
to_timestamp_in_ms = int(to_timestamp_in_s * 1000)

to_timestamp_in_ms = int(time() * 1000)

print(to_timestamp_in_ms)


1667235600000
1667259605054


In [38]:
# read data in a time window
values_battery = redis_client.ts().range(battery_string, from_timestamp_in_ms, to_timestamp_in_ms)
values_power = redis_client.ts().range(power_string, from_timestamp_in_ms, to_timestamp_in_ms)
values_plugged_seconds = redis_client.ts().range(plugged_seconds_string, from_timestamp_in_ms, to_timestamp_in_ms)

In [39]:
import pandas as pd

df_battery = pd.DataFrame(values_battery, columns = ['timestamp', 'battery_level'])
df_power = pd.DataFrame(values_power, columns = ['timestamp', 'power'])
df_plugged_seconds = pd.DataFrame(values_plugged_seconds, columns = ['timestamp', 'plugged_seconds'])

In [40]:
print("=== BATTERY INFO ===")
print("memory (Bytes): ", redis_client.ts().info(battery_string).memory_usage)
print("# samples: ", redis_client.ts().info(battery_string).total_samples)
print("# chunks: ", redis_client.ts().info(battery_string).chunk_count)
print()

df_battery

=== BATTERY INFO ===
memory (Bytes):  4184
# samples:  506
# chunks:  1



,timestamp,battery_level
0,1667259112688,65.0
1,1667259113818,65.0
2,1667259114952,65.0
3,1667259116076,65.0
4,1667259117212,65.0
...,...,...
427,1667259599517,68.0
428,1667259600642,68.0
429,1667259601761,68.0
430,1667259602882,68.0


In [41]:
DeepnoteChart(df_battery, """{"mark":{"type":"line","tooltip":true},"$schema":"https://vega.github.io/schema/vega-lite/v4.json","encoding":{"x":{"sort":null,"type":"nominal","field":"timestamp","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"nominal","field":"battery_level","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}},"height":440,"width":"container","autosize":{"type":"fit"},"config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

In [42]:
print("=== POWER INFO ===")
print("memory (Bytes): ", redis_client.ts().info(power_string).memory_usage)
print("# samples: ", redis_client.ts().info(power_string).total_samples)
print("# chunks: ", redis_client.ts().info(power_string).chunk_count)
print()

df_power

=== POWER INFO ===
memory (Bytes):  4240
# samples:  519
# chunks:  1



,timestamp,power
0,1667259112688,0.0
1,1667259113818,0.0
2,1667259114952,0.0
3,1667259116076,0.0
4,1667259117212,0.0
...,...,...
427,1667259599517,1.0
428,1667259600642,1.0
429,1667259601761,1.0
430,1667259602882,1.0


In [46]:
DeepnoteChart(df_power, """{"mark":{"type":"line","tooltip":true},"$schema":"https://vega.github.io/schema/vega-lite/v4.json","encoding":{"x":{"sort":null,"type":"quantitative","field":"timestamp","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"power","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}},"height":440,"width":"container","autosize":{"type":"fit"},"config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

In [47]:
print("=== PLUGGED SECONDS INFO ===")
print("memory (Bytes): ", redis_client.ts().info(plugged_seconds_string).memory_usage)
print("# samples: ", redis_client.ts().info(plugged_seconds_string).total_samples)
print("# chunks: ", redis_client.ts().info(plugged_seconds_string).chunk_count)
print()

df_plugged_seconds

=== PLUGGED SECONDS INFO ===
memory (Bytes):  216
# samples:  11
# chunks:  1



,timestamp,plugged_seconds
0,1667259060000,0.0
1,1667259120000,0.0
2,1667259180000,0.0
3,1667259240000,0.0
4,1667259300000,0.0
5,1667259360000,21.0
6,1667259420000,51.0
7,1667259480000,52.0
8,1667259540000,52.0
9,1667259600000,49.0


In [50]:
DeepnoteChart(df_plugged_seconds, """{"mark":{"type":"bar","tooltip":true},"$schema":"https://vega.github.io/schema/vega-lite/v4.json","encoding":{"x":{"sort":null,"type":"quantitative","field":"timestamp","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"plugged_seconds","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}},"height":440,"width":"container","autosize":{"type":"fit"},"config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1479b8a6-e5b3-45fe-9b0d-b80a7e9746ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>